In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import pickle
from experiment.batch_experiment import Batch

In [ ]:
batch_name = "batch-2022-02-18"
time_consistent_batch = Batch(batch_name, "exp_time_consistent")
time_inconsistent_batch = Batch(batch_name, "exp_time_inconsistent")

In [ ]:
time_consistent_batch.get_convergence_rate()

In [ ]:
time_inconsistent_batch.get_convergence_rate()

In [ ]:
time_consistent_batch.visualize_all_runs()

In [ ]:
time_inconsistent_batch.visualize_all_runs()

In [ ]:
time_consistent_batch.get_statistics()

In [ ]:
time_consistent_batch.info

In [ ]:
time_consistent_success_index = time_consistent_batch.data["experiment"][time_consistent_batch.data["is_converged"] == True].index
time_inconsistent_success_index = time_inconsistent_batch.data["experiment"][time_inconsistent_batch.data["is_converged"] == True].index

In [ ]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)
 
    if (a_set & b_set):
        return a_set & b_set
    else:
        print("No common elements")

In [ ]:
common_index = sorted(common_member(time_consistent_success_index, time_inconsistent_success_index))

In [ ]:
filtered_time_consistent_data = time_consistent_batch.data.iloc[common_index]
filtered_time_inconsistent_data = time_inconsistent_batch.data.iloc[common_index]

In [ ]:
filtered_time_consistent_data["iteration"].describe()

In [ ]:
filtered_time_inconsistent_data["iteration"].describe()

In [ ]:
filtered_time_consistent_data["iteration"].hist(bins=20)

In [ ]:
filtered_time_inconsistent_data["iteration"].hist(bins=20)